In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL
base_url = "https://aps.unmc.edu"
peptide_list_url = f"{base_url}/database/anti"
peptide_details_url = f"{base_url}/database/peptide"

# List to store peptide data
peptide_data = []

# Function to get the peptide details (including MIC values)
def get_peptide_details(peptide_id):
    data = {'ID': peptide_id}
    response = requests.post(peptide_details_url, data=data)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract details
    peptide_name = soup.find('h1', {'class': 'peptide'}).text.strip()
    
    # Extract all table data
    table_data = soup.find_all('td')
    
    peptide_info = {
        'Peptide ID': peptide_id,
        'Name': peptide_name,
        'MIC': None
    }
    
    # Find the MIC values from the "Activity" section
    mic_values = []
    for td in table_data:
        if 'Activity' in td.text:
            mic_text = td.find_next_sibling('td').text.strip()
            mic_values.append(mic_text)
            peptide_info['MIC'] = mic_text
    
    return peptide_info

# Function to scrape peptide IDs from the peptide list page
def scrape_peptide_ids():
    response = requests.get(peptide_list_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all form elements (each representing a peptide)
    forms = soup.find_all('form', {'action': '/database/peptide'})
    
    # Extract peptide IDs
    peptide_ids = []
    for form in forms:
        peptide_id = form.find('input', {'name': 'ID'})['value']
        peptide_ids.append(peptide_id)
    
    return peptide_ids

# Scrape all peptide IDs
peptide_ids = scrape_peptide_ids()

# Loop through each peptide ID and get the details
for peptide_id in peptide_ids:
    print(f"Scraping peptide: AP{peptide_id}")
    try:
        peptide_info = get_peptide_details(peptide_id)
        peptide_data.append(peptide_info)
        
        # Sleep to avoid overwhelming the server
        time.sleep(1)
    except Exception as e:
        print(f"Failed to scrape peptide AP{peptide_id}: {e}")

# Convert the scraped data to a DataFrame and save it to a CSV file
df = pd.DataFrame(peptide_data)
df.to_csv('peptide_mic_data.csv', index=False)
print("Scraping completed! Data saved to peptide_mic_data.csv.")


SSLError: HTTPSConnectionPool(host='aps.unmc.edu', port=443): Max retries exceeded with url: /database/anti (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))